In [1]:
# To support both python 2 and python 3
# 让这份笔记同步支持 python 2 和 python 3
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
#使用statsmodels便于统计分析数据
import statsmodels.api as sm
import statsmodels.formula.api as smf
#使用sklearn便于预测（机器学习）
from sklearn import linear_model
#使用patsy便于生成模型
import patsy

**12. Apply boosting, bagging, and random forests to a data set of your
choice. Be sure to fit the models on a training set and to evaluate their
performance on a test set. How accurate are the results compared
to simple methods like linear or logistic regression? Which of these
approaches yields the best performance?**

In [3]:
Weekly = pd.read_csv("data/Weekly.csv").dropna().drop('Unnamed: 0',axis = 1)
Weekly.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [4]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(Weekly, test_size = 1/3, random_state = 1)
columns = '1'
for column in Weekly.columns:
    columns = columns + '+' + column
y_test,X_test = patsy.dmatrices('Direction ~ {} - Direction - Year - Today'.format(columns), test_set, return_type = 'dataframe')
y_train,X_train = patsy.dmatrices('Direction ~ {} - Direction - Year - Today'.format(columns), train_set, return_type = 'dataframe')
X_train.drop('Intercept',axis = 1,inplace= True)
X_test.drop('Intercept',axis = 1,inplace= True)

In [5]:
#tree方法
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
tree_clf = DecisionTreeClassifier()
param_grid = [
{'max_leaf_nodes': np.arange(2,101)},
]
grid_search = GridSearchCV(tree_clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train,y_train['Direction[Up]'])
grid_search.best_params_

{'max_leaf_nodes': 14}

In [6]:
from sklearn.metrics import accuracy_score
tree_clf_predict = grid_search.predict(X_test)
accuracy_tree = accuracy_score(y_test['Direction[Up]'],tree_clf_predict)
accuracy_tree

0.5179063360881543

In [7]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train,y_train['Direction[Up]'])
lda_predict =lda.predict(X_test)
accuracy_lda = accuracy_score(y_test['Direction[Up]'],lda_predict)
accuracy_lda

0.5840220385674931

In [8]:
#QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X_train,y_train['Direction[Up]'])
qda_predict =qda.predict(X_test)
accuracy_qda = accuracy_score(y_test['Direction[Up]'],qda_predict)
accuracy_qda

0.5013774104683195

In [9]:
#Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(solver='liblinear')
log.fit(X_train,y_train['Direction[Up]'])
log_predict =log.predict(X_test)
accuracy_log = accuracy_score(y_test['Direction[Up]'],log_predict)
accuracy_log

0.5895316804407713

In [10]:
#bagging
from sklearn.ensemble import RandomForestClassifier
bag_rf_clf = RandomForestClassifier(n_estimators=300,random_state=1,max_features = len(X_train.columns))
bag_rf_clf.fit(X_train,y_train['Direction[Up]'])
bag_predict =bag_rf_clf.predict(X_test)
accuracy_bag = accuracy_score(y_test['Direction[Up]'],bag_predict)
accuracy_bag

0.5179063360881543

In [11]:
# random forests
rf_clf = RandomForestClassifier(n_estimators=300,random_state=1,max_features = 'sqrt')
rf_clf.fit(X_train,y_train['Direction[Up]'])
rf_predict = rf_clf.predict(X_test)
accuracy_rf = accuracy_score(y_test['Direction[Up]'],rf_predict)
accuracy_rf

0.509641873278237

In [12]:
#boosting
from sklearn.ensemble import GradientBoostingClassifier
boot_clf = GradientBoostingClassifier(n_estimators=5000,random_state=1,learning_rate=0.1)
boot_clf.fit(X_train,y_train['Direction[Up]'])
boot_predict = boot_clf.predict(X_test)
accuracy_boot = accuracy_score(y_test['Direction[Up]'],boot_predict)
accuracy_boot

0.5426997245179064

lda及logistic方案最佳